In [ ]:
# Cell 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Cell 2: Unzip Data from Drive to Local Colab Environment
import os

print("--- Step 1: Unzipping Dataset from Google Drive ---")

# --- USER: UPDATE THIS PATH ---
# This must be the exact path to the ZIP file you downloaded.
DRIVE_ZIP_PATH = '/content/drive/My Drive/DeepFakeDataset/140k-real-and-fake-faces.zip'
LOCAL_DATA_PATH = '/content/dataset'

# Clean up any old data and unzip the file to the fast local disk
if not os.path.exists(os.path.join(LOCAL_DATA_PATH, 'real_vs_fake')):
    print("Unzipping... this may take a minute.")
    !rm -rf "{LOCAL_DATA_PATH}"
    os.makedirs(LOCAL_DATA_PATH, exist_ok=True)
    !unzip -q "{DRIVE_ZIP_PATH}" -d "{LOCAL_DATA_PATH}"
else:
    print("Dataset already unzipped.")


print("✅ Data ready for training.\n")

--- Step 1: Unzipping Dataset from Google Drive ---
Dataset already unzipped.
✅ Data ready for training.



In [ ]:
# Cell 3: Define the ImageDataset Class
import torch
from torch.utils.data import Dataset
import cv2

class ImageDataset(Dataset):
    def __init__(self, data_list, transform=None):
        self.data_list = data_list
        self.transform = transform

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        img_path, label = self.data_list[idx]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if self.transform:
            img = self.transform(img)

        # CrossEntropyLoss requires labels as LongTensor
        return img, torch.tensor(label, dtype=torch.long)

print("ImageDataset class defined.")

ImageDataset class defined.


In [ ]:
# Cell 4: Define the SimpleCNN Model Architecture (with Dropout)
import torch.nn as nn

class SimpleCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32 * 56 * 56, 128)
        self.relu3 = nn.ReLU()

        # --- NEW: Added a Dropout layer to help prevent overfitting ---
        # It will randomly zero 50% of the elements during training
        self.dropout = nn.Dropout(0.5)

        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.flatten(x)
        x = self.relu3(self.fc1(x))

        # --- NEW: Apply dropout before the final layer ---
        x = self.dropout(x)

        x = self.fc2(x)
        return x

print("SimpleCNN model class defined.")

SimpleCNN model class defined.


In [ ]:
# Cell 5: Create the Data Loaders (with Augmentation)
import glob
from torchvision import transforms
from torch.utils.data import DataLoader

print("\n--- Step 2: Preparing Data Loaders ---")
DATA_DIR = '/content/dataset/real_vs_fake/real-vs-fake'
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
VALID_DIR = os.path.join(DATA_DIR, 'valid')

train_real_files = glob.glob(os.path.join(TRAIN_DIR, 'real', '*.jpg'))
train_fake_files = glob.glob(os.path.join(TRAIN_DIR, 'fake', '*.jpg'))
train_list = [(path, 0) for path in train_fake_files] + [(path, 1) for path in train_real_files]

valid_real_files = glob.glob(os.path.join(VALID_DIR, 'real', '*.jpg'))
valid_fake_files = glob.glob(os.path.join(VALID_DIR, 'fake', '*.jpg'))
valid_list = [(path, 0) for path in valid_fake_files] + [(path, 1) for path in valid_real_files]

print(f"Found {len(train_list)} training images.")
print(f"Found {len(valid_list)} validation images.")

im_size = 224
mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]

# --- CHANGED: Added data augmentation for the training set ---
# These random transformations help prevent overfitting
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((im_size, im_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# No augmentation for the validation set, just normalization
valid_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((im_size, im_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# --- CHANGED: Apply the respective transforms to the datasets ---
train_data = ImageDataset(train_list, transform=train_transforms)
valid_data = ImageDataset(valid_list, transform=valid_transforms)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_data, batch_size=32, shuffle=False, num_workers=2)

print("✅ Data loaders are ready.\n")


--- Step 2: Preparing Data Loaders ---
Found 100000 training images.
Found 20000 validation images.
✅ Data loaders are ready.



In [ ]:
# Cell 6: Train the Model with Checkpointing (30 Epochs)
import torch.optim as optim
from tqdm import tqdm
import os # Make sure os is imported

print("--- Step 3: Setting Up for Training ---")
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# --- Automatically determine save directory based on ZIP file location ---
DRIVE_ZIP_PATH = '/content/drive/My Drive/DeepFakeDataset/140k-real-and-fake-faces.zip'

# Get the directory where the ZIP file is located
DATASET_DIR = os.path.dirname(DRIVE_ZIP_PATH)
# Create a 'Models' folder inside that directory
MODEL_DIR = os.path.join(DATASET_DIR, 'Models')

# Define the full paths for the model files
BEST_MODEL_PATH = os.path.join(MODEL_DIR, 'simple_cnn_best_model.pth')
CHECKPOINT_PATH = os.path.join(MODEL_DIR, 'simple_cnn_checkpoint.pth')

# Ensure the 'Models' directory exists
os.makedirs(MODEL_DIR, exist_ok=True)
print(f"Models and checkpoints will be saved in: {MODEL_DIR}")


# Initialize model and optimizer
model = SimpleCNN().to(device)
lr = 1e-4

# --- CHANGED: Reduced the number of epochs ---
num_epochs = 30

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

# --- Load checkpoint if it exists --
# IMPORTANT: Ensure old checkpoint is deleted/moved for fresh training
start_epoch = 0
best_valid_acc = 0.0

if os.path.exists(CHECKPOINT_PATH):
    checkpoint = torch.load(CHECKPOINT_PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    best_valid_acc = checkpoint.get('best_valid_acc', 0.0)
    print(f"✅ Checkpoint found. Resuming training from epoch {start_epoch}")
else:
    print("ℹ️ No checkpoint found. Starting training from scratch.")

print(f"Training for {num_epochs} total epochs on device: {device}\n")

# --- Training loop starts from the last saved epoch ---
for epoch in range(start_epoch, num_epochs):
    # --- Training Phase ---
    model.train()
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    train_loss = running_loss / len(train_loader.dataset)
    train_acc = running_corrects.double() / len(train_loader.dataset)

    # --- Validation Phase ---
    model.eval()
    running_loss = 0.0
    running_corrects = 0

    with torch.no_grad():
        for inputs, labels in tqdm(valid_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Valid]"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

    valid_loss = running_loss / len(valid_loader.dataset)
    valid_acc = running_corrects.double() / len(valid_loader.dataset)

    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | Valid Loss: {valid_loss:.4f} Acc: {valid_acc:.4f}")

    # --- Save the best model based on validation accuracy ---
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        print(f"🎉 New best model saved with accuracy: {best_valid_acc:.4f}")

    # --- Save checkpoint after every epoch ---
    checkpoint_data = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'best_valid_acc': best_valid_acc
    }
    torch.save(checkpoint_data, CHECKPOINT_PATH)
    print(f"💾 Checkpoint saved for epoch {epoch+1}.\n")


print("--- TRAINING COMPLETE ---")

--- Step 3: Setting Up for Training ---
Models and checkpoints will be saved in: /content/drive/My Drive/DeepFakeDataset/Models
ℹ️ No checkpoint found. Starting training from scratch.
Training for 30 total epochs on device: cuda



Epoch 1/30 [Valid]: 100%|██████████| 625/625 [00:57<00:00, 10.91it/s]


Epoch 1/30 | Train Loss: 0.5728 Acc: 0.7005 | Valid Loss: 0.5050 Acc: 0.7636
🎉 New best model saved with accuracy: 0.7636
💾 Checkpoint saved for epoch 1.



Epoch 2/30 [Valid]: 100%|██████████| 625/625 [01:01<00:00, 10.16it/s]


Epoch 2/30 | Train Loss: 0.4960 Acc: 0.7605 | Valid Loss: 0.4392 Acc: 0.7993
🎉 New best model saved with accuracy: 0.7993
💾 Checkpoint saved for epoch 2.



...

Epoch 28/30 | Train Loss: 0.1764 Acc: 0.9281 | Valid Loss: 0.1514 Acc: 0.9421
🎉 New best model saved with accuracy: 0.9421
💾 Checkpoint saved for epoch 28.



Epoch 29/30 [Valid]: 100%|██████████| 625/625 [00:59<00:00, 10.44it/s]


Epoch 29/30 | Train Loss: 0.1709 Acc: 0.9303 | Valid Loss: 0.1437 Acc: 0.9481
🎉 New best model saved with accuracy: 0.9481
💾 Checkpoint saved for epoch 29.



Epoch 30/30 [Valid]: 100%|██████████| 625/625 [00:57<00:00, 10.89it/s]


Epoch 30/30 | Train Loss: 0.1672 Acc: 0.9327 | Valid Loss: 0.1461 Acc: 0.9455
💾 Checkpoint saved for epoch 30.

--- TRAINING COMPLETE ---


In [ ]:
# Cell 4: Define the SimpleCNN Model Architecture
import torch.nn as nn

class SimpleCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32 * 56 * 56, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.flatten(x)
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        return x

print("SimpleCNN model class defined.")


SimpleCNN model class defined.


In [ ]:
# Cell 5: Create the Data Loaders
import glob
from torchvision import transforms
from torch.utils.data import DataLoader

print("\n--- Step 2: Preparing Data Loaders ---")
DATA_DIR = '/content/dataset/real_vs_fake/real-vs-fake'
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
VALID_DIR = os.path.join(DATA_DIR, 'valid')

train_real_files = glob.glob(os.path.join(TRAIN_DIR, 'real', '*.jpg'))
train_fake_files = glob.glob(os.path.join(TRAIN_DIR, 'fake', '*.jpg'))
train_list = [(path, 0) for path in train_fake_files] + [(path, 1) for path in train_real_files]

valid_real_files = glob.glob(os.path.join(VALID_DIR, 'real', '*.jpg'))
valid_fake_files = glob.glob(os.path.join(VALID_DIR, 'fake', '*.jpg'))
valid_list = [(path, 0) for path in valid_fake_files] + [(path, 1) for path in valid_real_files]

print(f"Found {len(train_list)} training images.")
print(f"Found {len(valid_list)} validation images.")

im_size = 224
mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
data_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((im_size, im_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

train_data = ImageDataset(train_list, transform=data_transforms)
valid_data = ImageDataset(valid_list, transform=data_transforms)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_data, batch_size=32, shuffle=False, num_workers=2)

print("✅ Data loaders are ready.\n")


--- Step 2: Preparing Data Loaders ---
Found 100000 training images.
Found 20000 validation images.
✅ Data loaders are ready.



In [ ]:
# Cell 6: Train the Model with Checkpointing (Corrected Paths)
import torch.optim as optim
from tqdm import tqdm
import os # Make sure os is imported

print("--- Step 3: Setting Up for Training ---")
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# --- NEW: Automatically determine save directory based on ZIP file location ---
# This must be the exact path to the ZIP file you downloaded.
DRIVE_ZIP_PATH = '/content/drive/My Drive/DeepFakeDataset/140k-real-and-fake-faces.zip'

# Get the directory where the ZIP file is located
DATASET_DIR = os.path.dirname(DRIVE_ZIP_PATH)
# Create a 'Models' folder inside that directory
MODEL_DIR = os.path.join(DATASET_DIR, 'Models')

# Define the full paths for the model files
BEST_MODEL_PATH = os.path.join(MODEL_DIR, 'simple_cnn_best_model.pth')
CHECKPOINT_PATH = os.path.join(MODEL_DIR, 'simple_cnn_checkpoint.pth')

# Ensure the 'Models' directory exists
os.makedirs(MODEL_DIR, exist_ok=True)
print(f"Models and checkpoints will be saved in: {MODEL_DIR}")


# Initialize model and optimizer
model = SimpleCNN().to(device)
lr = 1e-4
num_epochs = 50
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

# --- Load checkpoint if it exists ---
start_epoch = 0
best_valid_acc = 0.0

if os.path.exists(CHECKPOINT_PATH):
    checkpoint = torch.load(CHECKPOINT_PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    best_valid_acc = checkpoint.get('best_valid_acc', 0.0)
    print(f"✅ Checkpoint found. Resuming training from epoch {start_epoch}")
else:
    print("ℹ️ No checkpoint found. Starting training from scratch.")

print(f"Training for {num_epochs} total epochs on device: {device}\n")

# --- Training loop starts from the last saved epoch ---
for epoch in range(start_epoch, num_epochs):
    # --- Training Phase ---
    model.train()
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    train_loss = running_loss / len(train_loader.dataset)
    train_acc = running_corrects.double() / len(train_loader.dataset)

    # --- Validation Phase ---
    model.eval()
    running_loss = 0.0
    running_corrects = 0

    with torch.no_grad():
        for inputs, labels in tqdm(valid_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Valid]"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

    valid_loss = running_loss / len(valid_loader.dataset)
    valid_acc = running_corrects.double() / len(valid_loader.dataset)

    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | Valid Loss: {valid_loss:.4f} Acc: {valid_acc:.4f}")

    # --- Save the best model based on validation accuracy ---
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        print(f"🎉 New best model saved with accuracy: {best_valid_acc:.4f}")

    # --- Save checkpoint after every epoch ---
    checkpoint_data = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'best_valid_acc': best_valid_acc
    }
    torch.save(checkpoint_data, CHECKPOINT_PATH)
    print(f"💾 Checkpoint saved for epoch {epoch+1}.\n")


print("--- TRAINING COMPLETE ---")

--- Step 3: Setting Up for Training ---
Models and checkpoints will be saved in: /content/drive/My Drive/DeepFakeDataset/Models
✅ Checkpoint found. Resuming training from epoch 33
Training for 50 total epochs on device: cuda



Epoch 34/50 [Valid]: 100%|██████████| 625/625 [01:03<00:00,  9.85it/s]


Epoch 34/50 | Train Loss: 0.0090 Acc: 0.9969 | Valid Loss: 0.2659 Acc: 0.9466
💾 Checkpoint saved for epoch 34.



Epoch 35/50 [Valid]: 100%|██████████| 625/625 [00:58<00:00, 10.60it/s]


Epoch 35/50 | Train Loss: 0.0094 Acc: 0.9967 | Valid Loss: 0.2783 Acc: 0.9454
💾 Checkpoint saved for epoch 35.



...

Epoch 48/50 | Train Loss: 0.0060 Acc: 0.9979 | Valid Loss: 0.3307 Acc: 0.9415
💾 Checkpoint saved for epoch 48.



Epoch 49/50 [Valid]: 100%|██████████| 625/625 [00:59<00:00, 10.43it/s]


Epoch 49/50 | Train Loss: 0.0057 Acc: 0.9982 | Valid Loss: 0.2874 Acc: 0.9469
💾 Checkpoint saved for epoch 49.



Epoch 50/50 [Valid]: 100%|██████████| 625/625 [00:59<00:00, 10.49it/s]


Epoch 50/50 | Train Loss: 0.0049 Acc: 0.9983 | Valid Loss: 0.2988 Acc: 0.9466
💾 Checkpoint saved for epoch 50.

--- TRAINING COMPLETE ---
